# 7. FICC Trading

FICC는 Fixed Income, Currency, Commodity의 약자로 전통적인 자산 중 주식을 제외한 자산들을 일컫는 말입니다. 이전 세션에서 주식 파생상품의 헤지트레이딩을 실시한 것처럼, 이번 세션에서는 시장에 존재하는 몇 가지의 FICC 파생상품과 그 운용 방법을 공부합니다.

In [262]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy import optimize

## 1. IRS with YTM

저희는 이자율에 대해 배울 때, IRS에 대해 공부했으며 그 때의 IRS는 매 기마다 91일물 CD금리와 고정 금리를 교환하는 상품이었습니다. 하지만 시장에 존재하는 이자율은 매우 많기 때문에, 91일물 CD금리가 아닌 금리를 변동금리로 삼는 IRS 또한 많이 존재합니다. 한 예시로, 국고채 3년 YTM 금리를 변동금리로 삼는 IRS가 존재합니다. 이런 식으로 무이표금리가 아닌 YTM을 변동금리로 삼는 IRS를 평가하는 방법은 다음과 같습니다.
1. 채권의 발행일이 지금보다 $t$년 이후이고, 만기가 $T$년이며, 연당 이자 지금 횟수 $n$회, 이표율 $r$을 가지는 가상채권을 설정합니다.(이 때 $T$와 $n$은 기준이 되는 채권과 동일하게 설정합니다)
2. 주어진 현물이자율에서 선도금리를 계산하여 해당 가상채권의 가격을 구합니다.
3. 해찾기를 통해 액면가와 채권가격이 동일해지는 $r$을 구합니다.<br>

변동 금리를 지급하는 주체는 $r$을 매 기마다 지급하고, 고정금리를 지급하는 주체는 고정금리를 지급하는 것을 이용해 이자율스왑의 가치를 평가합니다.<br>
우선 선형 보간 함수와 선도금리 계산 함수, 현물이자율 계산 함수를 작성합니다.

In [268]:
def linear_interpolation(Tr,r,T):
    if T<=Tr[0]:
        rt = r[0]
    elif T>=Tr[-1]:
        rt = r[-1]
    else:
        MaxL = np.where(Tr<=T)[0][-1]
        MinU = np.where(Tr>T)[0][0]
        RL = r[MaxL]
        RU = r[MinU]
        rt = RL + ((RU-RL)/(Tr[MinU]-Tr[MaxL]))*(T-Tr[MaxL])
    return rt
def Calc_ForwardRate(Tr,r,start,period): # 현물이자율이 주어졌을 때, (start, start+period)동안 적용되는 선도금리 계산
    fend = start + period
    DF1 = np.exp(-linear_interpolation(Tr,r,start)*start)
    DF2 = np.exp(-linear_interpolation(Tr,r,fend)*fend)
    forward = ((DF1/DF2)-1)/period
    return forward

def bondprice(last,index,Tr,ZR,ytm):
    ZR2 = ZR.copy()
    ZR2[index] = last
    bond = 0
    NC = int(Tr[index]*2)
    for i in range(0,NC):
        tenor = (i+1)*0.5
        zr = linear_interpolation(Tr[0:index+1],ZR2[0:index+1],tenor)
        bond+=ytm*np.exp(-zr*tenor)/2
    bond+=np.exp(-zr*tenor)
    return bond-1

def YTMtoZero(Tr,r):
    N = len(r)
    zero = r.copy()
    zero*=0
    for i in range(0,N):
        if Tr[i]<1:
            zero[i] = r[i]
        else:
            rt = optimize.fsolve(bondprice,0.03,args = (i,Tr,zero,r[i]))
            zero[i] = rt
    return zero

In [235]:
Tr[0] = 0.25
Tr[1] = 0.5
Tr[2] = 0.75
Tr[3] = 1
Tr[4] = 1.5
Tr[5] = 2
Tr[6] = 2.5
Tr[7] = 3
Tr[8] = 4
Tr[9] = 5
Tr[10] = 7
Tr[11] = 10
Tr[12] = 15
Tr[13] = 20
Tr[14] = 30
Tr[15] = 50
r[0] = 0.03065
r[1] = 0.0294
r[2] = 0.02925
r[3] = 0.02860
r[4] = 0.02840
r[5] = 0.02830
r[6] = 0.02825
r[7] = 0.02772
r[8] = 0.02847
r[9] = 0.02820
r[10] = 0.02945
r[11] = 0.02920
r[12] = 0.02875
r[13] = 0.02870
r[14] = 0.02820
r[15] = 0.02737

In [259]:
ZR = YTMtoZero(Tr,r)

그리고 현물이자율과 가상채권의 발행일, 만기, 이자 지급 횟수, 이표율이 정해졌을 때 채권의 가격을 구하는 함수를 작성합니다.

In [289]:
def VBond(coupon,Tr,ZR,Start,Mat,couponT):
    NC = int(Mat/couponT)
    bond = 0
    for i in range(0,NC):
        CS = Start + i*couponT
        FR = Calc_ForwardRate(Tr,ZR,Start,CS+couponT)
        bond+= coupon*couponT*np.exp(-FR*(CS+couponT))
    bond+= np.exp(-FR*(Start+Mat))
    PR = linear_interpolation(Tr,ZR,Start)
    bond*=np.exp(Start*PR)
    return bond-1

이자율을 변경하는 시점에서, 채권 가격이 액면가와 같아지는 이표율이 그 시점에서의 YTM이 되고, 변동 금리 지급 주체는 해당 금리를 지급합니다. <br>

이제 한 예시로, 액면가 10000원, 국고채 3년 YTM과 고정금리를 6년동안 3개월마다 교환하는 이자율스왑의 가치평가를 실행해 보겠습니다. 고정금리는 4%로 가정합니다. 우선 매 기의 현금흐름을 구합니다.

In [302]:
IRS = pd.DataFrame(index = ["Float","Fixed"],columns = np.arange(0.25,6.25,0.25))
IRS.loc["Fixed"]=0.04

In [303]:
for i in range(0,IRS.shape[1]):
    start = IRS.columns[i]
    ytm =  optimize.fsolve(VBond,0.03,args = (Tr,ZR,start,3,0.5))
    IRS.iloc[0,i] = ytm

그리고 변동금리의 현금흐름에서 고정금리의 현금흐름을 뺀 것을 현재가로 할인한 가치의 합을 구하면 됩니다.

In [304]:
VIRS = 0
for i in range(0,IRS.shape[1]):
    Dr = linear_interpolation(Tr,ZR,(i+1)*0.25)
    VIRS += 10000*(IRS.iloc[0,i]-IRS.iloc[1,i])*np.exp(-(i+1)*0.25*Dr)

In [305]:
VIRS

-777.736813533069

## 2. Bond Forward

본드 포워드는 말 그대로 채권 선도계약입니다. 미리 정해진 가격에 채권을 거래하기로 약정하는 장외파생상품입니다. 장기채를 매도자가 구입하고, 보유하고 있다가 약정한 시기에 약정한 가격으로 매수자에게 장기채를 매도합니다. 매도자 관점에서 현금흐름은 다음과 같습니다.
<br>

채권 발행일 : -(채권가격)

<br>

채권 발행일 ~ 채권선도 결제시점 : +(이표)

<br>

채권선도 결제시점 : +(선도가)

<br>

이 중 본드 포워드의 가치평가에 사용되는 현금흐름은 선도가입니다. 채권선도 결제시점에서의 채권의 가격과 선도가의 차이를 현재가로 할인한 것이 본드 포워드의 가치가 됩니다.

In [308]:
def BondForward(ForwardT,Forward,ZR,Mat,coupon,couponT):
    BondPrice = 0
    CFN = int((Mat-ForwardT)/couponT)
    for i in range(0,CFN):
        FR = Calc_ForwardRate(Tr,ZR,ForwardT,couponT*(i+1))
        BondPrice+=coupon*couponT*(FR*(couponT*(i+1)))
    BondPrice += np.exp(FR*(Mat-ForwardT))
    Profit = ForwardT-BondPrice
    ZR = linear_interpolation(Tr,ZR,ForwardT)
    return Profit*np.exp(-ZR*ForwardT)

In [310]:
BondForward(5,0.8,ZR,30,0.03,0.5)

1.6779422340142127

## 3. PV01 and Interest Rate Hedging

이자율 파생상품의 운용을 위해 알아야 하는 민감도는 PV01입니다. PV01은 한 이자율 파생상품의 기준금리가 한 tenor에서만 변했을 때, 파생상품의 가치가 변하는 민감도를 의미합니다. 즉,
$$PV01(t_1) = {{V(r_1) - V(r_2)}\over 2dr}$$
$s.t$
$r_1 = \begin{cases} 
		r(t) & \text{if }t\neq t_1 \\ 
        r(t)+dr & \text{if }t = t_1 
     \end{cases}$        $r_2 = \begin{cases} 
		r(t) & \text{if }t\neq t_1 \\ 
        r(t)-dr & \text{if }t = t_1 
     \end{cases}$<br>
     
주식 파생상품에서 델타를 0으로 만들며 헤지 트레이딩을 했던 것처럼 금리 파생상품은 이 PV01을 0으로 만들며 헤지 트레이딩을 합니다. 하지만 PV01은 테너마다 존재하기에 무한히 생성이 가능하고, 따라서 0으로 만들 PV01을 지정해야 한다는 특징이 있으며, 헤징을 위해 채권을 매수/매도해야 하는데 이표채도 테너마다 PV01이 존재하기에 원하는 한 개의 PV01만 조작할 수 없다는 것이 또 하나의 어려운 점입니다.

In [323]:
def BFPV01(ForwardT,Forward,ZR,Mat,coupon,couponT,index):
    ZR1 = ZR.copy()
    ZR2 = ZR.copy()
    ZR1[index] += 0.0001
    ZR2[index] -= 0.0001
    BF1 = BondForward(ForwardT,Forward,ZR1,Mat,coupon,couponT)
    BF2 = BondForward(ForwardT,Forward,ZR2,Mat,coupon,couponT)
    return (BF1-BF2)/0.0002

In [326]:
for i in range(0,len(ZR)):
    print(Tr[i])
    print(BFPV01(5,0.8,ZR,30,0.03,0.5,i))

0.25
0.0
0.5
0.0
0.75
0.0
1.0
0.0
1.5
0.0
2.0
0.0
2.5
0.0
3.0
0.0
4.0
0.0
5.0
19.9323145337138
7.0
-0.5170462442927182
10.0
-1.327727183040306
15.0
-2.634039808797617
20.0
-6.907472028884198
30.0
-148.7815388224134
50.0
0.0


테너별 PV01을 보면, 본드포워드의 가치는 채권선도 결제시점~만기 의 금리에 대한 함수인 것을 확인할 수 있습니다.